In [ ]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
from collections import Counter

In [ ]:
import spacy

In [3]:
file_path = "data/Tweets.csv"

In [4]:
df = pd.read_csv('data/Tweets.csv')

In [5]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [6]:
print(f"Data has {df.shape[0]} rows and {df.shape[1]} columns.")

Data has 14640 rows and 15 columns.


In [7]:
sub_df = df[['tweet_id','airline_sentiment','airline','text']]

In [8]:
# columns with NA values
sub_df.isna().sum()

tweet_id             0
airline_sentiment    0
airline              0
text                 0
dtype: int64

In [9]:
df[['airline_sentiment']].value_counts()

airline_sentiment
negative             9178
neutral              3099
positive             2363
Name: count, dtype: int64

In [10]:
df[['airline']].value_counts()

airline       
United            3822
US Airways        2913
American          2759
Southwest         2420
Delta             2222
Virgin America     504
Name: count, dtype: int64

In [11]:
NER = spacy.load("en_core_web_sm")

In [12]:
def pre_process(text: str):
    return re.sub(r"@","",text)

In [13]:
df['text'] = df['text'].apply(pre_process)

In [14]:
df.head(5)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,VirginAmerica What dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,VirginAmerica plus you've added commercials to...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,VirginAmerica I didn't today... Must mean I ne...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,VirginAmerica it's really aggressive to blast ...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,VirginAmerica and it's a really big bad thing ...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [15]:
def find_entity(text: str):
    NER_dict = dict()
    text = NER(text)
    for word in text.ents:
        NER_dict[word.text] = word.label_
        
    return NER_dict

In [22]:
df['entities'] = df['text'].apply(find_entity)

In [24]:
df[['entities']]

,entities
0,{}
1,{'VirginAmerica': 'ORG'}
2,{'today': 'DATE'}
3,{'VirginAmerica': 'ORG'}
4,{'VirginAmerica': 'ORG'}
...,...
14635,{'Chicago': 'GPE'}
14636,"{'AmericanAir': 'ORG', 'over 20 minutes': 'TIM..."
14637,"{'American Airlines': 'ORG', 'BlackBerry10': '..."
14638,{'AmericanAir': 'ORG'}


In [25]:
# we will try to collect the information in 'entities', by iterating through all the rows
# 'entity_dict' will store 'entity_i' as a key, and all examples of that entity as a list of values

In [31]:
entity_dict = dict()
for index, row in df.iterrows():
    current_entities = row['entities']
    for k,v in current_entities.items():
        try:
            entity_dict[v].append(k)
        except KeyError:
            entity_dict[v]=[k]

In [33]:
entity_dict.keys()

dict_keys(['ORG', 'DATE', 'MONEY', 'TIME', 'ORDINAL', 'CARDINAL', 'PERSON', 'GPE', 'PRODUCT', 'WORK_OF_ART', 'NORP', 'LAW', 'LOC', 'QUANTITY', 'PERCENT', 'FAC', 'EVENT', 'LANGUAGE'])

In [38]:
l = [1,2,3,4,5]
s_df = pd.DataFrame(l).add_prefix("_ABC")

In [41]:
s_df = pd.DataFrame('FC')

NameError: name 'NEG_Decision' is not defined